In [12]:
import pandas as pd
import os
import kenpompy
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Returns an authenticated browser that can then be used to scrape pages that require authorization.
# Credentials are loaded from environment variables (stored in .env file)



True

In [10]:
import requests
import json

# API Configuration
# API key is loaded from environment variables (stored in .env file)
BASE_URL = "https://kenpom.com/api.php"
API_KEY = os.getenv('KENPOM_API_KEY')

headers = {
    "Authorization": f"Bearer {API_KEY}"
}

print("=" * 70)
print("KenPom API Demo - How to Use")
print("=" * 70)

# Example 1: Get Current Ratings for 2025 Season (Top 10 Teams)
print("\n1. CURRENT RATINGS - Top 10 Teams (2025 Season)")
print("-" * 70)
response = requests.get(
    f"{BASE_URL}?endpoint=ratings&y=2025",
    headers=headers
)
data = response.json()
import pandas as pd
data = response.json()
df = pd.DataFrame(data)



KenPom API Demo - How to Use

1. CURRENT RATINGS - Top 10 Teams (2025 Season)
----------------------------------------------------------------------


In [11]:
df

,DataThrough,Season,TeamName,Seed,ConfShort,Coach,Wins,Losses,AdjEM,RankAdjEM,...,RankNCSOS,Event,APL_Off,RankAPL_Off,APL_Def,RankAPL_Def,ConfAPL_Off,RankConfAPL_Off,ConfAPL_Def,RankConfAPL_Def
0,End of season,2025,Duke,1,ACC,Jon Scheyer,35,4,39.2864,1,...,21,NCAA,17.13,112,19.01,362,17.29,5,19.26,18
1,End of season,2025,Houston,1,B12,Kelvin Sampson,35,5,36.5890,2,...,100,NCAA,19.69,354,18.62,349,20.10,16,18.72,16
2,End of season,2025,Florida,1,SEC,Todd Golden,36,4,36.4571,3,...,237,NCAA,16.28,41,17.67,194,16.93,4,17.49,12
3,End of season,2025,Auburn,1,SEC,Bruce Pearl,32,6,35.0506,4,...,19,NCAA,17.35,139,17.54,163,17.46,11,17.29,7
4,End of season,2025,Tennessee,2,SEC,Rick Barnes,30,8,30.9286,5,...,202,NCAA,18.44,280,18.74,355,18.76,16,18.45,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,End of season,2025,Alabama A&M,0,SWAC,Otis Hughley,10,22,-23.3581,360,...,118,None,16.21,31,17.06,59,16.60,4,16.82,2
360,End of season,2025,Coppin St.,0,MEAC,Larry Stewart,6,24,-24.3953,361,...,66,None,17.33,136,17.30,97,16.59,3,17.33,6
361,End of season,2025,Chicago St.,0,NEC,Scott Spinelli,4,28,-24.3999,362,...,29,None,17.62,174,17.00,50,18.02,4,17.93,2
362,End of season,2025,Arkansas Pine Bluff,0,SWAC,Solomon Bozeman,6,25,-25.9429,363,...,39,None,15.99,23,17.09,61,15.97,1,17.35,7
